In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

##### Data

In [3]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image,label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000  #10000 samples are taken at once while shuffling

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))


### Model
##### Outline the model

In [10]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                           ])

##### Choose the optimizer and the loss fucntion

In [11]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

##### Training 

In [12]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2, validation_steps=10)

Epoch 1/5
540/540 - 15s - loss: 0.3274 - accuracy: 0.9069 - val_loss: 0.1710 - val_accuracy: 0.9493
Epoch 2/5
540/540 - 13s - loss: 0.1416 - accuracy: 0.9585 - val_loss: 0.1142 - val_accuracy: 0.9623
Epoch 3/5
540/540 - 13s - loss: 0.0973 - accuracy: 0.9704 - val_loss: 0.0861 - val_accuracy: 0.9738
Epoch 4/5
540/540 - 13s - loss: 0.0746 - accuracy: 0.9770 - val_loss: 0.0754 - val_accuracy: 0.9763
Epoch 5/5
540/540 - 13s - loss: 0.0614 - accuracy: 0.9814 - val_loss: 0.0565 - val_accuracy: 0.9817


##### Testing the model

In [13]:
test_loss, test_accuracy = model.evaluate(test_data)

      1/Unknown - 2s 2s/step - loss: 0.0797 - accuracy: 0.9759

In [14]:
print("Test loss: {0:.2f}. Test accuracy: {1:2f}%".format(test_loss, test_accuracy*100.))

Test loss: 0.08. Test accuracy: 97.589999%
